## Module for building Henderson's Mixed Model Equations for single trait including marker  and polygenic effect



### Data set to test the module

In [1]:
using DataFrames
using Distributions

In [2]:
FILE = "../../data/small.ped";

In [11]:
Animal = ["S1","D1","O1","O3"]
y = [100.0, 50.0, 150.0, 40.0];
df = DataFrame(Animal=Animal,y=y);

In [12]:
srand(123)
d = Binomial(2,0.5)
nObs     = 4
nMarkers = 5
M        = float(rand(d,(nObs,nMarkers)));

In [13]:
df=[df DataFrame(M)]

,Animal,y,x1,x2,x3,x4,x5
1,S1,100.0,1.0,0.0,1.0,1.0,1.0
2,D1,50.0,2.0,0.0,2.0,2.0,1.0
3,O1,150.0,1.0,2.0,0.0,1.0,0.0
4,O3,40.0,0.0,0.0,2.0,1.0,1.0


### <font color="red"> Run module</font>

In [14]:
MODULE_PATH="../../Module/MME.jl";

In [15]:
include(MODULE_PATH);

In [16]:
ped = PedModule.mkPed(FILE);

In [41]:
mme = MMEModule.initMME("y = intercept + Animal")
varg=1.0*0.5
vare=5.0
λ1 = varg/vare
G = reshape([λ1],1,1)
MMEModule.setAsRandom(mme,"Animal",ped,G)

In [42]:
resG = MMEModule.getSolG(mme,df,outFreq=100)

6x2 Array{Any,2}:
 "intercept: intercept"  84.7563     
 "Animal: S1"             2.27265    
 "Animal: D1"            -2.27295    
 "Animal: O1"             3.1067     
 "Animal: O3"            -2.1314     
 "Animal: O2"            -0.000149061

In [40]:
mme.Gi

1x1 Array{Float64,2}:
 0.1

### add marker information

In [19]:
MMEModule.addMarkers(mme,df[:,3:7]);

In [32]:
vec(full(mme.ySparse))

4-element Array{Float64,1}:
 100.0
  50.0
 150.0
  40.0

In [24]:
lhs=full(mme.mmeLhs)

6x6 Array{Float64,2}:
 4.0   1.0    1.0    1.0   1.0   0.0
 1.0   2.25   0.75  -0.5  -0.5  -0.5
 1.0   0.75   2.25  -0.5  -0.5  -0.5
 1.0  -0.5   -0.5    2.0   0.0   0.0
 1.0  -0.5   -0.5    0.0   2.0   0.0
 0.0  -0.5   -0.5    0.0   0.0   1.0

In [25]:
M=mme.M

4x5 Array{Float64,2}:
 1.0  0.0  1.0  1.0  1.0
 2.0  0.0  2.0  2.0  1.0
 1.0  2.0  0.0  1.0  0.0
 0.0  0.0  2.0  1.0  1.0

In [30]:
vRes = 1.0
vEff = 1.0*0.5/5 #c
λ=vRes/vEff

10.0

In [29]:
full(mme.X'mme.X)

6x6 Array{Float64,2}:
 4.0  1.0  1.0  1.0  1.0  0.0
 1.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [32]:
lhs2=[lhs mme.X'M
    M'mme.X M'M+eye(size(M,2))*λ]

11x11 Array{Float64,2}:
 4.0   1.0    1.0    1.0   1.0   0.0   4.0   2.0   5.0   5.0   3.0
 1.0   2.25   0.75  -0.5  -0.5  -0.5   1.0   0.0   1.0   1.0   1.0
 1.0   0.75   2.25  -0.5  -0.5  -0.5   2.0   0.0   2.0   2.0   1.0
 1.0  -0.5   -0.5    2.0   0.0   0.0   1.0   2.0   0.0   1.0   0.0
 1.0  -0.5   -0.5    0.0   2.0   0.0   0.0   0.0   2.0   1.0   1.0
 0.0  -0.5   -0.5    0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0
 4.0   1.0    2.0    1.0   0.0   0.0  16.0   2.0   5.0   6.0   3.0
 2.0   0.0    0.0    2.0   0.0   0.0   2.0  14.0   0.0   2.0   0.0
 5.0   1.0    2.0    0.0   2.0   0.0   5.0   0.0  19.0   7.0   5.0
 5.0   1.0    2.0    1.0   1.0   0.0   6.0   2.0   7.0  17.0   4.0
 3.0   1.0    1.0    0.0   1.0   0.0   3.0   0.0   5.0   4.0  13.0

In [34]:
y=mme.ySparse
rhs=[mme.X'y
     M'y]

11x1 Array{Float64,2}:
 340.0
 100.0
  50.0
 150.0
  40.0
   0.0
 350.0
 300.0
 280.0
 390.0
 190.0

In [37]:
sol = fill(0.0,size(lhs2,2));
res1=MMEModule.Gibbs(lhs2,sol,rhs,50000,outFreq=10000)

at sample: 10000
at sample: 20000
at sample: 30000
at sample: 40000

11-element Array{Float64,1}:
  89.9693    
  14.728     
 -14.7234    
  25.1934    
 -17.7709    
   0.00488876
   0.671616  
   5.03858   
  -5.40019   
  -1.10396   
  -2.51771   


at sample: 50000


In [154]:
sol = fill(0.0,size(lhs,1));
MMEModule.GaussSeidel(lhs,sol,rhs)

10 

11-element Array{Float64,1}:
  84.9652 
   5.09672
 -11.4662 
  20.1023 
 -13.7329 
   0.0    
   3.00111
   2.74656
  -2.29328
  -2.74666
   1.01918

0.00026709575586159853
